In [32]:
from transformers import AutoTokenizer, AutoModel
import torch
import os
import PyPDF2
from PyPDF2 import PdfReader
import pandas as pd
import re
import sys
# Load LEGAL-BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("pile-of-law/legalbert-large-1.7M-2")
model = AutoModel.from_pretrained("pile-of-law/legalbert-large-1.7M-2")

# # Prepare text for embedding generation
# text = "Example legal text to embed that is required so now i want to do this?"
# inputs = tokenizer(text, return_tensors="pt")

# # Generate embeddings
# with torch.no_grad():
#     outputs = model(**inputs)

# # Retrieve the embeddings from the last hidden state
# embeddings = outputs

# embeddings now contains the generated embeddings for your text


In [27]:
dataset_path = os.path.join(os.path.dirname(os.getcwd()), 'Dataset')
csv_path = os.path.join(os.path.dirname(os.getcwd()),'res','csv_etl_files')
pdf_paths = os.listdir(dataset_path)
csv_data = os.listdir(csv_path)
vector_path = os.path.join(os.path.dirname(os.getcwd()),'res','Vector_Embedding')

In [30]:
print(csv_data)

['A2015-22.csv', 'A2016-18_0.csv', 'A2019-21.csv', 'A2017-27_1.csv', 'A2019-20.csv', 'A2017-23_0.csv', 'The Insolvency and Bankruptcy Code, 2016..csv', 'A2016-38_1.csv', 'A2017-26_1.csv', 'A2016-4_1.csv', 'A2017-02_2.csv', 'A2018-03.csv', 'A2018-17.csv', 'A2017-20_0.csv', 'A2016-2_0.csv', 'A2017-22_1.csv', 'A2017-13_1.csv', 'A2018-02_0.csv', 'A2018-24_0.csv', 'A2016-11_1.csv', 'A2018-04_0.csv', 'A2016-49_1.csv', 'A2018-25.csv', 'A2017-15_2.csv', 'A2016-17_1.csv', 'A2019-17.csv', 'A2016-36_0.csv', 'A2017-10_3.csv', 'A2016-16_0.csv', 'A2016-30_3.csv', 'A2015-11.csv', 'A2017-12_5.csv', 'A2019-10.csv', 'A2016-32_0.csv', 'A2017-14_1.csv', 'Indian Institutes of Management Act 2017.csv', 'A2017-16_2.csv']


In [44]:
from tqdm import tqdm
for i in tqdm(range(len(csv_data))):
    print(csv_data[i])
    csv_full_path = csv_path+"/"+csv_data[i]
    df = pd.read_csv(csv_full_path)
    tensor_data = None
    for j in tqdm(range(len(df['Paragraph Text']))):
        txt = df['Paragraph Text'].iloc[j]
        inputs = tokenizer(txt,return_tensors='pt')
        if(inputs['input_ids'].shape[1]>512):
            zeros = torch.zeros_like(embeddings)
            tensor_data = torch.concat((tensor_data,zeros),dim=0)
            continue
            
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:,0,:]
        if tensor_data is None:
            tensor_data = embeddings
        else:
            tensor_data = torch.concat((tensor_data,embeddings),dim=0)
        tensor_path = vector_path+"/"+csv_data[i][:-3]+'pt'
        torch.save(tensor_data,tensor_path)
        

  0%|                                                    | 0/37 [00:00<?, ?it/s]

A2015-22.csv



 55%|██████████████████████▍                  | 112/205 [00:18<00:12,  7.45it/s]


  3%|█▏                                          | 1/37 [00:30<18:07, 30.20s/it]

A2016-18_0.csv


...

Indian Institutes of Management Act 2017.csv



 89%|████████████████████████████████████▎    | 109/123 [00:14<00:01, 10.03it/s]


 97%|█████████████████████████████████████████▊ | 36/37 [13:45<00:25, 25.46s/it]

A2017-16_2.csv



100%|███████████████████████████████████████████| 37/37 [13:58<00:00, 22.66s/it]


In [12]:
embeddings.pooler_output.shape

torch.Size([1, 768])

In [13]:
print(embeddings.last_hidden_state.shape)

torch.Size([1, 19, 768])
